In [1]:
import pandas as pd
final_df = pd.read_csv(r"C:\Users\Anuj Bohra\Desktop\ArogoAI\Severity\combined_cleaned_df_final.csv")

In [2]:
final_df = final_df.drop('Unnamed: 0', axis=1)
from sklearn.preprocessing import LabelEncoder

gender_encoder = LabelEncoder()

final_df['gender'] = gender_encoder.fit_transform(final_df['gender'])
df_icd9 = final_df[final_df['icd_version'] == 9]

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
icd_encoded = encoder.fit_transform(df_icd9[['icd_code']])
icd_encoded_df = pd.DataFrame(
    icd_encoded,
    columns=encoder.get_feature_names_out(['icd_code'])
)
X = df_icd9.drop(['hadm_id', 'icd_code', 'subject_id'], axis=1)
y = icd_encoded_df
y_labels = y.idxmax(axis=1)
X.columns = X.columns.astype(str)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X)

import numpy as np
y_class = np.argmax(y.values, axis=1)

In [6]:
%pip install torchvision


   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.8/172.3 MB 16.3 MB/s eta 0:00:11
   ---------------------------------------- 1.7/172.3 MB 17.8 MB/s eta 0:00:10
    --------------------------------------- 2.6/172.3 MB 18.4 MB/s eta 0:00:10
    --------------------------------------- 3.5/172.3 MB 18.3 MB/s eta 0:00:10
   - -------------------------------------- 4.4/172.3 MB 19.8 MB/s eta 0:00:09
   - -------------------------------------- 5.2/172.3 MB 19.6 MB/s eta 0:00:09
   - -------------------------------------- 6.1/172.3 MB 19.4 MB/s eta 0:00:09
   - -------------------------------------- 7.1/172.3 MB 19.6 MB/s eta 0:00:09
   - -------------------------------------- 8.5/172.3 MB 19.3 MB/s eta 0:00:09
   -- ------------------------------------- 9.3/172.3 MB 19.3 MB/s eta 0:00:09
   -- ------------------------------------- 10.3/172.3 MB 19.2 MB/s eta 0:00:09
   -- ------------------------------------- 11.2/172.3 MB 

In [ ]:
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self, input_size=25, hidden1=256, hidden2=512, hidden3=256, hidden4=128, num_classes=1203):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2, hidden3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden3, hidden4)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(hidden4, num_classes)
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.relu3(self.fc3(x))
        x = self.relu4(self.fc4(x))
        x = self.fc5(x)  # Raw logits (will apply softmax later)
        return x

# ------------------------------------------------------------------------------
# 2. Load the pre-trained models
# ------------------------------------------------------------------------------

# Load XGBoost model from a pickle file
xgb_model_path = r'C:\Users\Anuj Bohra\Desktop\ArogoAI\Severity\Model\xgb_model.pkl'  # Update with your actual file path
loaded_xgb = joblib.load(xgb_model_path)

# Load PyTorch model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pytorch_model_path = r'C:\Users\Anuj Bohra\Desktop\ArogoAI\Severity\Model\simple_net_state_dict.pth'  # Update with your actual file path
pytorch_model = SimpleNet(input_size=25, num_classes=1203).to(device)
pytorch_model.load_state_dict(torch.load(pytorch_model_path, map_location=device))
pytorch_model.eval()

# ------------------------------------------------------------------------------
# 3. Define the ensemble prediction functions
# ------------------------------------------------------------------------------

def ensemble_predict(X, weights=None):
    """
    Returns the ensemble's top-1 prediction for each sample.
    This is useful for computing overall accuracy.
    
    Args:
        X (np.array): Input feature array of shape (n_samples, n_features)
        weights (list or tuple): Weights for the two models [w_xgb, w_pt].
                                 Defaults to equal weighting if None.
    
    Returns:
        final_preds (np.array): Array of shape (n_samples,) with the predicted class labels.
    """
    if weights is None:
        weights = [1/2, 1/2]
    w_xgb, w_pt = weights

    # XGBoost: Get probability distribution
    xgb_probs = loaded_xgb.predict_proba(X)  # Shape: (n_samples, num_classes)
    
    # PyTorch: Get probability distribution (apply softmax to logits)
    X_tensor = torch.from_numpy(X).float().to(device)
    with torch.no_grad():
        outputs = pytorch_model(X_tensor)
        pt_probs = F.softmax(outputs, dim=1).cpu().numpy()  # Shape: (n_samples, num_classes)
    
    # Compute weighted sum of probabilities
    ensemble_probs =  w_xgb * xgb_probs + w_pt * pt_probs

    # Return the class with the highest probability for each sample
    final_preds = np.argmax(ensemble_probs, axis=1)
    return final_preds

def ensemble_topk(X, top_k=3, weights=None):
    """
    Returns the top-k predictions (class indices and their probabilities) for each sample.
    
    Args:
        X (np.array): Input feature array of shape (n_samples, n_features)
        top_k (int): Number of top predictions to return.
        weights (list or tuple): Weights for the two models [w_xgb, w_pt].
                                 Defaults to equal weighting if None.
    
    Returns:
        top_k_indices (np.array): Array of shape (n_samples, top_k) with top-k class indices.
        top_k_probs (np.array): Array of shape (n_samples, top_k) with corresponding probabilities.
    """
    if weights is None:
        weights = [1/2, 1/2]
    w_xgb, w_pt = weights

    # Get probability distributions from each model
    xgb_probs = loaded_xgb.predict_proba(X)  # Shape: (n_samples, num_classes)
    X_tensor = torch.from_numpy(X).float().to(device)
    with torch.no_grad():
        outputs = pytorch_model(X_tensor)
        pt_probs = F.softmax(outputs, dim=1).cpu().numpy()  # Shape: (n_samples, num_classes)

    # Compute weighted sum of probabilities
    ensemble_probs =  w_xgb * xgb_probs + w_pt * pt_probs

    # For each sample, retrieve the indices of the top k probabilities.
    top_k_indices = np.argsort(ensemble_probs, axis=1)[:, -top_k:][:, ::-1]
    top_k_probs = np.take_along_axis(ensemble_probs, top_k_indices, axis=1)
    
    return top_k_indices, top_k_probs

# ------------------------------------------------------------------------------
# 4. Example usage
# ------------------------------------------------------------------------------

# Assuming X_scaled is your NumPy array of input features:
# For instance:
# X_scaled = np.load('X_scaled.npy')

# # Get top-1 predictions (for accuracy computation)
# final_predictions = ensemble_predict(X_scaled, weights=[0.4, 0.3, 0.3])
# print("Final predictions (top-1) for each sample:\n", final_predictions)

# # # Get top-3 predictions (with probabilities)
# top_k_indices, top_k_probs = ensemble_topk(X_scaled, top_k=3, weights=[0.4, 0.3, 0.3])
# print("Top-3 class indices for each sample:\n", top_k_indices)
# print("Corresponding probabilities for top-3 predictions:\n", top_k_probs)


C:\Users\Anuj Bohra\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [13:29:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [ ]:
# Get top-1 predictions (for accuracy computation)
final_predictions = ensemble_predict(X_scaled, weights=[0.7,0.3])

In [9]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_class, final_predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.96
